In [ ]:
using CausalityTools
include("../entropy.jl")
using Random
using BenchmarkTools

rng = MersenneTwister(145)

a = rand(rng, 0:10, 100)
b = rand(rng, 0:10, 100)

est = Kraskov(k=1)

print("Current method :")
foo = @btime TE(Int.(a .> 0), Int.(b .> 0))
println("The result is $foo.\n")


print("True transfer entropy :")
a = float.(a)
b = float.(b)
foo2 = @btime transferentropy(a, b, est)
println("The result is $foo2.\n")

print("CCM :")
foo3 = @btime crossmap(a, b, 2, 1)
println("The result is $foo3.\n")

In [ ]:
import PyPlot as plt

rng = MersenneTwister(145)

a = rand(rng, 0:10, 100)
b = rand(rng, 0:10, 100)

a = float.(a)
b = float.(b)

Ls = [10:5:50; 60:10:100]

@btime begin
    test = [crossmap(a[1:L], b[1:L], 2, 1) for L in Ls]
    test2 = [crossmap(b[1:L], a[1:L], 2, 1) for L in Ls]
end

plt.figure()
plt.plot(Ls, test, "b-", label="a to b")
plt.plot(Ls, test2, "r-", label="b to a")
plt.legend()
show(plt.gcf())

In [ ]:
using CausalityTools

s_measure()

In [14]:
using CausalityTools

# A two-dimensional Ulam lattice map
sys = ulam(2)

# Sample 1000 points after discarding 5000 transients
orbit = trajectory(sys, 1000, Ttr = 5000)
x, y = orbit[:, 1], orbit[:, 2]

# 4-dimensional embedding for `x`, 5-dimensional embedding for `y`
s_measure(x, y, dx = 4, τx = 3, dy = 5, τy = 1)

MethodError: MethodError: no method matching s_measure(::Vector{Float64}, ::Vector{Float64}; dx=4, τx=3, my=5, τy=1)
Closest candidates are:
  s_measure(::AbstractVector{T}, ::AbstractVector{T}; K, dx, dy, τx, τy, metric, tree_metric) where T at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:140 got unsupported keyword argument "my"
  s_measure(!Matched::DelayEmbeddings.AbstractDataset{D}, ::AbstractVector{T}; K, dy, τy, metric, tree_metric) where {D, T} at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:155 got unsupported keyword arguments "dx", "τx", "my"
  s_measure(::AbstractVector{T}, !Matched::DelayEmbeddings.AbstractDataset{D}; K, dx, τx, metric, tree_metric) where {D, T} at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:163 got unsupported keyword arguments "my", "τy"

In [23]:
using DelayEmbeddings

a = collect(1:100)
p = embed(a, 2, 5)

2-dimensional Dataset{Int64} with 95 points
  1    6
  2    7
  3    8
  4    9
  5   10
  6   11
  7   12
  8   13
  9   14
 10   15
  ⋮  
 87   92
 88   93
 89   94
 90   95
 91   96
 92   97
 93   98
 94   99
 95  100

In [26]:
using Neighborhood

a = [0, 1.2, 0.1, 0.8]
b = [0, 0, 1.8, 0.8]


X = embed(a, 2, 1)
Y = embed(b, 2, 1)

treeX = searchstructure(KDTree, X, Euclidean())
treeY = searchstructure(KDTree, Y, Euclidean())
neighborhoodtype, theiler = NeighborNumber(1), Theiler(0)
idxs_X = bulkisearch(treeX, X, neighborhoodtype, theiler)
idxs_Y = bulkisearch(treeY, Y, neighborhoodtype, theiler)

print(idxs_Y)

LoadError: UndefVarError: embed not defined

In [3]:
using CausalityTools
using BenchmarkTools

include("../Utils/entropy.jl")

x = rand(0:10, 50)
y = rand(0:10, 50)

@btime s_measure(float(x), float(y), K=3, dx=5, dy=5, τx=1, τy=1)

@btime TE(Int.(x .> 0), Int.(y .> 0))

  39.459 μs (689 allocations: 48.64 KiB)
  1.271 μs (11 allocations: 1.23 KiB)


NaN

In [11]:
using HypothesisTests

x = rand(0:10, 50)
y = rand(0:10, 50)

@btime jdd(OneSampleTTest, x, y, B=10, D=2, τ=1)

  58.584 μs (65 allocations: 222.17 KiB)


One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0.0
    point estimate:          0.275
    95% confidence interval: (-0.0137, 0.5637)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.0596

Details:
    number of observations:   10
    t-statistic:              2.1547879226714413
    degrees of freedom:       9
    empirical standard error: 0.12762276839711598


In [36]:
source = collect(1:5)
target = collect(11:15)

D = 2
B = 10
τs = 1

@btime Ex = DelayEmbeddings.embed(source, D, τs)
Ey = DelayEmbeddings.embed(target, D, τs)
Mx = DelayEmbeddings.Matrix(Ex)
My = DelayEmbeddings.Matrix(Ey)

  890.733 ns (7 allocations: 304 bytes)


4×2 Matrix{Int64}:
 11  12
 12  13
 13  14
 14  15

In [34]:
import DelayEmbeddings

source = collect(1:5)
target = collect(11:15)

D = 2
B = 10
τ = 1

js = ([1 for i = 1:D]...,)
τs = (collect(0:-τ:-(D-1)*τ)...,)

@btime Ex = DelayEmbeddings.genembed(source, τs, js)
Ey = DelayEmbeddings.genembed(target, τs, js)
Mx = DelayEmbeddings.Matrix(Ex)
My = DelayEmbeddings.Matrix(Ey)

  96.062 ns (3 allocations: 192 bytes)


4×2 Matrix{Int64}:
 12  11
 13  12
 14  13
 15  14

In [33]:
foo = [0 1; 2 0]
LinearIndices(foo[foo .> 0])

2-element LinearIndices{1, Tuple{Base.OneTo{Int64}}}:
 1
 2

In [24]:
My

4×2 Matrix{Int64}:
 12  11
 13  12
 14  13
 15  14

In [19]:
Mx

4×2 Matrix{Int64}:
 2  1
 3  2
 4  3
 5  4

In [21]:
using Distances

pairwise(Euclidean(), Mx, Mx, dims=1)

4×4 Matrix{Float64}:
 0.0      1.41421  2.82843  4.24264
 1.41421  0.0      1.41421  2.82843
 2.82843  1.41421  0.0      1.41421
 4.24264  2.82843  1.41421  0.0

In [23]:
Euclidean()([2,1], [3,2])

1.4142135623730951

In [81]:
B = 10
d = 3
τ = 1
alpha = 0.01

K = 3
dx = 3
dy = 3
τx = 1
τy = 1

cuttoff = 0.5
cuttoff2 = 0.05

func1(x, y) = pvalue(jdd(OneSampleTTest, x, y, B=B, D=d, τ=τ, μ0=0.0), tail=:right) < alpha ? 1 : 0
func2(x, y) = TE(Int.(x .> 0), Int.(y .> 0)) > cuttoff ? 1 : 0
func3 = s_measure(float(x), float(y), K=K, dx=dx, dy=dy, τx=τx, τy=τy) > cuttoff2 ? 1 : 0

tot1 = 0
tot2 = 0
tot3 = 0
nans = 0
for i = 1:100
    x = rand(0:10, 100)
    y = rand(0:10, 100)

    tot1 += func1(x ,y)
    tot2 += func2(x ,y)
    tot2 += func2(x ,y)
end

println("jdd : $tot1")
println("TE : $tot2")
println("s measure : $tot3")

jdd : 26
TE : 0
s measure : 0


In [73]:
a = collect(1:100)
b = collect(101:200)
func(a, b)

4.969036725783444e-19

In [82]:
func2(x, y) = TE(Int.(x .> 0), Int.(y .> 0))

nans = 0
for i = 1:100
    x = rand(0:10, 100)
    y = rand(0:10, 100)

    isnan(func2(x,y)) ? nans += 1 : nans += 0
end

nans

100